In [1]:
from database.GraphDB import KnowledgeGraphDB
from retrieval.retrieval_hub import RetrievalHub
from query_and_ranking.query import Query
from query_and_ranking.rerank import Reranker
from agentic import Agent

2024-06-15 15:21:53.383144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 15:21:53.383184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 15:21:53.435224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 15:21:54.460478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

neo4j_uri = os.getenv('NEO4J_URI')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [3]:
kg = KnowledgeGraphDB(uri=neo4j_uri, user=neo4j_user, password=neo4j_password)
retrieval_hub = RetrievalHub(kg)
reranker = Reranker()
cv_query = Query(kg, retrieval_hub)

In [4]:
chat_agent = Agent(kg, retrieval_hub, reranker, cv_query)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import gradio as gr
from processing_data.read_pdf import *

def chat(input_text, history):
    response = chat_agent.chat(input_text)
    
    # For now
    text = response['text']
    if 'files' in response:
        text += '\n\n' + '\n'.join(response['files'])
    print(text)
    history.append((input_text, text))
    return text

def chat_multi(message, history):
    num_files = len(message["files"])
    if num_files >0:
        yield "Processing {} files...".format(num_files)
        data  = process_pdf_batch(message["files"], is_save=False)
        text = '\n\n'.join(data)
        input_text = message["text"]+ '\n\n' + text
    else:
        input_text = message["text"]
    response = chat_agent.chat(input_text)
    
    # For now
    text = response['text']
    if 'files' in response:
        text += '\n\n' + '\n'.join(response['files'])
    history.append((input_text, text))
    return text, history

In [6]:
# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(height=300)
#     msg  = gr.Textbox(lines=2, label="Prompt", placeholder="Type here")
#     btn = gr.Button("Submit")
#     clear = gr.ClearButton(components=[msg, chatbot], value="Clear")
    
#     btn.click(chat, inputs=[msg, chatbot], outputs=[msg, chatbot])
#     msg.submit(chat, inputs=[msg, chatbot], outputs=[msg, chatbot])
# demo.launch()

gr.ChatInterface(
    chat,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Give me a question to query the CV", container=False, scale=7),
    title="CV Query Chatbot",
    description="A LLMs with Knowledge Graph Chatbot to query the CVs",
    theme="soft",
    examples=["MBBank is looking for 25 candidates who are currently studying or graduated in National Economics University. Can you find all of them?","""Viettel Digital Talent want 6 young students or fresh graduated, having enthusiastic passion on Data Science and AI
Requirements:

- Candidate must located in HaNoi
- Proficiency in Python
- Having projects related to DS/AI is a plus"""],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

Using cache from '/home/quanghung20gg/Documents/hung20gg/cv_rank/gradio_cached_examples/15' directory. If method or examples have changed since last caching, delete this folder to clear cache.

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
chat_agent.chat("Hi how are you?")

{'text': "Hello, I'm doing well, thank you for asking. I am an HR assistant and I'm here to provide you with high-quality and precise responses to your queries. Please feel free to ask me anything, and if I'm unsure about anything, I'll be sure to ask for clarification. If I don't have the answer, I'll let you know."}

In [7]:
prompt ="""
Viettel Digital Talent want 6 young students or fresh graduated, having enthusiastic passion on Data Science and AI 
Requirements:
- Candidate must located in HaNoi
- Proficiency in Python 
- Having projects related to DS/AI is a plus
"""

In [5]:
chat_agent.routing_response(prompt)

You are not running the flash-attention implementation, expect numerical differences.


True

In [9]:
neu = """ 
MBBank wants 25 candidates who are currently studying or graduated in National Economics University.
"""
chat_agent.routing_response(neu)

True

In [10]:
chat_agent.get_cv_from_jd(neu,llm_ranking=True)

Extracting Job Description


End LLM
New Query:  {'institutionList': ['national economics university (neu)'], 'majorList': ['finance', 'mathematical economics', 'commerce', 'public finance', 'finance and banking', 'socio-economic statistics', 'information technology and digital transformation in economics', 'economics and biology'], 'eduList': ['master', 'undergraduate', 'bachelor', 'diploma', 'certificate', 'phd']}
Extended Num Application:  30
Not enough application
New Query:  {'institutionList': ['university of economics', 'national economics university (neu)']}
[1214, 1274, 1690, 424, 463, 8, 1941, 2013, 495, 324, 227, 1783, 310, 561, 732, 406, 606, 280, 738, 124, 1761, 1993, 2109, 1722, 1291]
Num CV ok:  25
Rerank
[1214, 124, 1274, 1291, 1690, 1722, 1761, 1783, 1941, 1993, 2013, 2109, 227, 280, 310, 324, 406, 424, 463, 495, 561, 606, 732, 738, 8]
Time: 4.814815044403076


{'text': 'Understood. Here is the normalized job description based on the given text:\n\nNumber of applications required: Specific number (25)\nLocation: Not mentioned\nAcademic level: Undergraduate or graduate\nInstitution: National Economics University\nGPA: Not mentioned\nSpeaking language: Not mentioned\n\nRoles: Not mentioned\nSkills: Not mentioned\nProgramming languages and tech stacks: Not mentioned\nExperience level: Not mentioned\n\nOther requirements: Currently studying or graduated from National Economics University\n\nBenefits: Not mentioned\n\nSuitable roles: Not mentioned (as the actual role is not specified in the text)',
 'id': [1214,
  124,
  1274,
  1291,
  1690,
  1722,
  1761,
  1783,
  1941,
  1993,
  2013,
  2109,
  227,
  280,
  310,
  324,
  406,
  424,
  463,
  495,
  561,
  606,
  732,
  738,
  8],
 'files': ['data/raw/data/clean4.0/CV_SonBao_DS.pdf',
  'data/raw/data/clean4.0/CV_ Hoàng Phương Thảo - Thảo Hoàng.pdf',
  'data/raw/data/clean4.0/CV thương mại điệ

In [13]:
kg.query("Match (n:Application)-[r]->(m:Role) where id(n) = 1579 return  m, r.exp, type(r)")

[{'m': {'name': ' web developer'}, 'r.exp': 2.5, 'type(r)': 'ROLE'},
 {'m': {'name': 'back-end developer'}, 'r.exp': 2.5, 'type(r)': 'ROLE'},
 {'m': {'name': 'back-end developer'}, 'r.exp': None, 'type(r)': 'SUITABLE'},
 {'m': {'name': 'software engineer'}, 'r.exp': 2.5, 'type(r)': 'ROLE'},
 {'m': {'name': 'software engineer'}, 'r.exp': None, 'type(r)': 'SUITABLE'},
 {'m': {'name': 'full stack developer'}, 'r.exp': None, 'type(r)': 'SUITABLE'},
 {'m': {'name': 'java developer'}, 'r.exp': None, 'type(r)': 'SUITABLE'}]

In [8]:
rank_prompt = """
Can you rank them 
"""


In [9]:
retrieval_hub.cache.db_cache_role.similarity_search_with_relevance_scores("Marketing Manager", 5)

[(Document(page_content='marketing manager'), 0.9999999999999063),
 (Document(page_content='marketing staff'), 0.7674812526354313),
 (Document(page_content='marketing team leader'), 0.7667723637169023),
 (Document(page_content='marketing assistant'), 0.7459399351412406),
 (Document(page_content='sales manager'), 0.7358898083795753)]

In [12]:
0.7*1.05


0.735

In [5]:
from database.utils.GraphDB_tools import graph_to_text
hung = graph_to_text(kg.kg, 995)
print(hung)

Languages:
- English
Majors:
- bachelor, data science, national economic university, (GPA: 3.88)
Work:
- neu itlab (1 years)
- viettel digital talent (0.25 years)
Roles:
- research assistant (1 years)
- machine learning engineer (0.5 years)
- artificial intelligence engineer (0.5 years)
- data scientist (0.5 years)
Skills:
- knowledge graphs
- large language models
- natural language processing
- web technology
- data mining
- machine learning
- deep learning
- computer vision
Programming languages:
- python (1.5 years)
- c++ (1.5 years)
- javascript (1 years)
- sql (1 years)
- tensorflow (1 years)
- pytorch (1 years)
- transformers (0.5 years)
- diffusers (0.5 years)
Cities:
- ha noi
Certifications:
- IELTS 7.0 (2021)
Awards:
- IELTS 7.0 (2021)
- Third prize in ICPC Regional (non-specialized category)
- Second prize in the National Student Olympiads in Informatics (non-specialized category)
- Excellent scholarship 2/2 semesters



In [9]:
chat_agent.routing_ranking_response(rank_prompt)

True

In [10]:
chat_agent.chat(rank_prompt)

Based on the provided job description and the candidate CVs, I have ranked them according to the mentioned criteria. Here's the ranking along with the rationale:

```json
[
    {
        "id": 1007,
        "rank": 1,
        "rationale": "This candidate fits the job description perfectly. They have a bachelor's degree in data science, relevant skills like machine learning, deep learning, natural language processing, and proficiency in Python. Additionally, they have experience as a research assistant, machine learning engineer, AI engineer, and data scientist. Their certifications, awards, and academic achievements further strengthen their candidacy."
    },
    {
        "id": 1476,
        "rank": 2,
        "rationale": "This candidate has a bachelor's degree in data science and AI, relevant skills like AWS, ReactJS, Flutter, and proficiency in multiple programming languages including Python, Java, and PyTorch. Their experience in software engineering, AI, and data science, as well

{'text': 'Based on the provided job description and the candidate CVs, I have ranked them according to the mentioned criteria. Here\'s the ranking along with the rationale:\n\n```json\n[\n    {\n        "id": 1007,\n        "rank": 1,\n        "rationale": "This candidate fits the job description perfectly. They have a bachelor\'s degree in data science, relevant skills like machine learning, deep learning, natural language processing, and proficiency in Python. Additionally, they have experience as a research assistant, machine learning engineer, AI engineer, and data scientist. Their certifications, awards, and academic achievements further strengthen their candidacy."\n    },\n    {\n        "id": 1476,\n        "rank": 2,\n        "rationale": "This candidate has a bachelor\'s degree in data science and AI, relevant skills like AWS, ReactJS, Flutter, and proficiency in multiple programming languages including Python, Java, and PyTorch. Their experience in software engineering, AI, 

In [19]:
chat_agent.chat("Can you show me the candidates id 1944")

'Certainly, here are the details for the candidate with ID 1944:\n\n**ID: 1944**\n\n**Summary:**\nA highly skilled and experienced data scientist with a strong background in machine learning, deep learning, and natural language processing. Possesses in-depth knowledge in building and deploying large-scale AI systems, with a focus on knowledge graphs, language models, and deep neural networks. Passionate about leveraging cutting-edge AI technologies to drive impactful solutions.\n\n**Education:**\n- Bachelor\'s degree in Computer Science, Hanoi University of Science and Technology\n- GPA: 3.92/4.0\n- Awarded "Excellent Student Scholarship" for outstanding academic performance\n\n**Work Experience:**\n- Data Scientist, VinAI Research (2 years)\n  - Developed large-scale knowledge graph systems for text understanding\n  - Designed and implemented deep learning models for natural language processing\n  - Optimized and deployed models on cloud-based infrastructures\n- Machine Learning Inter

In [11]:
chat_agent.chat("Can you summarize our conversation?")

"Certainly, let me summarize our conversation:\n\nYou asked me to provide helpful responses as an HR assistant for a recruitment process happening in May 2024. Viettel is looking to hire 5 fresher software engineers in Hanoi, Vietnam. The requirements are:\n\n- Undergraduate or Bachelor's degree in Computer Science or a related field\n- At least 6 months of experience with Python, Java, C++, or NodeJS\n\nYou provided me with 5 candidate CVs that matched the job description, and asked me to rank them based on the given information.\n\nI reviewed the candidate profiles and provided a ranked list with explanations:\n\n1. ID: 1476 - Ranked 1st due to the strong technical skills, relevant work experience, impressive academic achievements, and awards/hackathon wins.\n\n2. ID: 1747 - Ranked 2nd with a good educational background, relevant work experience, and well-aligned technical skills.\n\n3. ID: 999 - Ranked 3rd with a decent educational background, substantial work experience, and some r

In [12]:
s = "Certainly, let me summarize our conversation:\n\nYou asked me to provide helpful responses as an HR assistant for a recruitment process happening in May 2024. Viettel is looking to hire 5 fresher software engineers in Hanoi, Vietnam. The requirements are:\n\n- Undergraduate or Bachelor's degree in Computer Science or a related field\n- At least 6 months of experience with Python, Java, C++, or NodeJS\n\nYou provided me with 5 candidate CVs that matched the job description, and asked me to rank them based on the given information.\n\nI reviewed the candidate profiles and provided a ranked list with explanations:\n\n1. ID: 1476 - Ranked 1st due to the strong technical skills, relevant work experience, impressive academic achievements, and awards/hackathon wins.\n\n2. ID: 1747 - Ranked 2nd with a good educational background, relevant work experience, and well-aligned technical skills.\n\n3. ID: 999 - Ranked 3rd with a decent educational background, substantial work experience, and some relevant technical skills.\n\n4. ID: 754 - Ranked 4th with a relevant educational background and some work experience, but lacking in certain technical skills.\n\n5. ID: 131 - Ranked 5th as the candidate is from Ho Chi Minh City, which does not match the job location requirement in Hanoi.\n\nYou then asked me to provide more details on the candidate with ID 131, which I summarized, noting their educational background, work experience, and technical skills.\n\nFinally, you asked me to summarize our entire conversation, which is what I have done here.\n\nPlease let me know if I have missed or misunderstood anything in the summary."
print(s)

Certainly, let me summarize our conversation:

You asked me to provide helpful responses as an HR assistant for a recruitment process happening in May 2024. Viettel is looking to hire 5 fresher software engineers in Hanoi, Vietnam. The requirements are:

- Undergraduate or Bachelor's degree in Computer Science or a related field
- At least 6 months of experience with Python, Java, C++, or NodeJS

You provided me with 5 candidate CVs that matched the job description, and asked me to rank them based on the given information.

I reviewed the candidate profiles and provided a ranked list with explanations:

1. ID: 1476 - Ranked 1st due to the strong technical skills, relevant work experience, impressive academic achievements, and awards/hackathon wins.

2. ID: 1747 - Ranked 2nd with a good educational background, relevant work experience, and well-aligned technical skills.

3. ID: 999 - Ranked 3rd with a decent educational background, substantial work experience, and some relevant technica